In [1]:
import sys
import importlib
sys.path.append('../../')

import lxml.html as html

import knutils.autoscraper as kas

In [ ]:
importlib.reload(kas)

In [2]:
root = html.parse('../../../DataSets/SiteSample/S1/List.html')

In [3]:
#set labels

sample = []
target_urls = []

hit_end = False
for x in root.iter():
    sample.append(x)
    if x.tag == 'a':
        url = x.attrib['href']
        if 'index.php' in url:
            hit_end = True
            
        if ('pejnya.net' in url) and ('content' in url) and not hit_end:
            target_urls.append(x)

In [4]:
depth_stats, tag_stats, attrk_stats, attrv_stats, attrkv_stats = kas.AutoScraper.get_stats(root)

p_max_depth = max([d for (d, n) in depth_stats if n > 1])
p_used_tags = [x for (x, n) in tag_stats if type(x) is str]
for t in ['br', 'b']:
    p_used_tags.remove(t)
t_used_attrk = ['class', 'id', 'name', 'title', 'type']
t_used_attrv_raw = [z for (k, v) in attrkv_stats.items() if k in t_used_attrk for (z, n) in v]
t_used_attrv = list(set(t_used_attrv_raw))
p_used_attrkv = [(k, v) for k in t_used_attrk for v in t_used_attrv if v in [x[0] for x in attrkv_stats[k]]]

p_context = [('cur', lambda p: p),
             ('prev-1', lambda p: kas.AutoScraper.get_sibling(p, -1)),('next-1', lambda p: kas.AutoScraper.get_sibling(p, +1)),
            ('child-0',lambda p: kas.AutoScraper.get_child(p, 0)),
             ('child-1',lambda p: kas.AutoScraper.get_child(p, 1)),
             ('child-2',lambda p: kas.AutoScraper.get_child(p, 2))]

def custom_order(clsf_name, split, split_stat):
    res = kas.SplitStat.calc_gini(clsf_name, split, split_stat)
    
    if clsf_name=='iter':
        if split.base().tag_name() == 'other':
            res *= 0.1
    elif clsf_name=='tags':
        if split.base().base().tag_name()=='other':
            res *= 0.1
            
    if clsf_name in ('tags', 'attrs'):
        if split.base().name() != 'cur':
            res *= 0.1
    
    return res

In [12]:
classifier1 = kas.ParentRelClassifier(kas.ContextClassifier(kas.TagClassifier(p_used_tags), p_context), 5)
classifier2 = kas.ParentRelClassifier(kas.ContextClassifier(kas.AttribClassifier(p_used_attrkv), p_context), 5)
classifier3 = kas.FwdIterClassifier(kas.TagClassifier(p_used_tags), 20)
fin_classifier = kas.AutoScraper({'tags':classifier1, 'attrs':classifier2, 'iter':classifier3})

In [13]:
%%time
scraper = fin_classifier.fit(root, target_urls, order_function=custom_order)

Wall time: 3.36 s


In [14]:
scraper.get_split_tree()

(('iter', <img> at fwd-iter-id=0, 636-0 vs 60-8),
 (('attrs', class=txt-main at context cur at rel-depth=1, 60-0 vs 0-8),
  None,
  None),
 None)

In [8]:
res = scraper.select(root)

[True, True, True, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, True, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, True, False, False, False, False, False, True, False, True, False, True, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, True, False, True, False, True, False, True, False, True, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, 

In [11]:
for x in res:
    print(x.attrib['href'])

http://pejnya.net/content/chastnoe.php?news=prislannoe_837
http://pejnya.net/content/chastnoe.php?news=prislannoe_836
http://pejnya.net/content/chastnoe.php?news=prislannoe_835
http://pejnya.net/content/photo.php?news=podborochka_241
http://pejnya.net/content/photo.php?news=vspominaya_leto_71
http://pejnya.net/content/photo.php?news=popi_170
http://pejnya.net/content/photo.php?news=podb_chastnih_ero_photo_665
http://pejnya.net/content/photo.php?news=podb_chastnih_ero_photo_664


In [ ]:
[x for (x,_) in tag_stats if type(x) is str] - ['br', 'b']

In [ ]:
p_used_tags.remove('br')

In [ ]:
p_used_tags